<a href="https://colab.research.google.com/github/LiviaAniely/CorrectBot/blob/main/CorrectBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports necessários

!pip install openai
!pip install python-dotenv

!pip install PyPDF2

from PyPDF2 import PdfReader

import gdown
from google.colab import drive

import os
import openai
from dotenv import load_dotenv, find_dotenv

import panel as pn

In [ ]:
# Importar o documento pdf contendo o resumo da cartilha de redação, com os critérios de correção

url = "https://drive.google.com/uc?id=1m9YrtR-X4yuZmLigsmETOCHkwHG4xbRy"
gdown.download(url, quiet=True, use_cookies=True)

'cartilha de redação.pdf'

In [ ]:
#Leitura do documento e passagem do texto para uma variável

# Objeto PdfReader
reader = PdfReader('cartilha de redação.pdf')

criterios = ""

for i in range(len(reader.pages)):
  criterios += reader.pages[i].extract_text()


print(len(criterios))

5027


In [ ]:
# Configuração da API

# Importar documento com a chave da API
drive.mount('/content/drive', force_remount=True)
file_path = '/content/drive/MyDrive/key.txt'

with open(file_path, 'r') as file:
    API_KEY = file.readline().strip()

_ = load_dotenv(find_dotenv())
openai.api_key  = API_KEY

Mounted at /content/drive


In [ ]:
# Função para receber a resposta de uma mensagem
def perguntar_ao_chat(messages, model="gpt-3.5-turbo"):
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

In [ ]:
# Função para coletar as mensagens do usuário e retornar as respostas
def coletar_mensagens(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})

    response = perguntar_ao_chat(context)
    context.append({'role':'assistant', 'content':f"{response}"})

    panels.append(
        pn.Row('Usuário:', pn.pane.Markdown(prompt, width=620, styles={'background-color': '#E6E6FA', 'border': '2px solid black', 'border-radius': '10px', 'text-align': 'justify', 'margin': '1px'})))
    panels.append(
        pn.Row('CorrectBot:', pn.pane.Markdown(response, width=620, styles={'background-color': '#FFE4E1', 'border': '2px solid black', 'border-radius': '10px', 'text-align': 'justify', 'margin': '1px'})))

    return pn.Column(*panels)

In [ ]:
# Permitir a conversa entre usuário e o bot através de um painel
pn.extension()

panels = []

context = [ {'role':'system', 'content':f"""
Você é o CorrectBot, um serviço automatizado para corrigir redações do Enem:exame nacional do ensino médio, e
dar uma nota de acordo com os critérios de correção desse modelo de redação.\
Para funcionar com sucesso, esses são os passos que você deve seguir:\
1° Apresente-se e pergunte o tema da redação ao usuário.\
2° Pergunte qual o texto da redação.\
3° Analise-o de acordo com os critérios de correção que estão no seguinte texto entre aspas simples: '{criterios}'.
É importante saber que cada competência varia de 40 em 40 pontos, começando de 0 até 200 pontos no total, ou seja, cada competência só pode ter as seguintes
notas:0,40,80,120,160 ou 200. As 5 competências contabilizam 1000 pontos.\
4° Por fim, diga qual é a nota da redação, que varia de 0 a 1000.\
Além disso, também dê a nota de cada uma das 5 competências.\
A sua resposta deve conter no máximo 1000 palavras.
"""} ]

inp = pn.widgets.TextInput(value="Caixa de resposta", placeholder='Digite sua resposta aqui...')
button_conversation = pn.widgets.Button(name="Enviar")

interactive_conversation = pn.bind(coletar_mensagens, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=2500),
)

dashboard

Column
    [0] TextInput(placeholder='Digite sua resposta a...)
    [1] Row
        [0] Button(name='Enviar')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=2500, loading_indicator=True)

# Análise

Para a análise da ferramenta, foram feitos testes com redações reais. Foram usadas um total de 15 redações de 5 temas diferentes com notas originais variando entre 680 e 1000 pontos.

Sendo assim, analisamos a diferença entre a nota original e a nota proposta pelo CorrectBot, e obtivemos uma média de 224 pontos de diferença entre ambas, com uma moda de 200 pontos.

Nesse sentido, é válido mencionar que a correção de redações por si só é uma tarefa subjetiva, esta ferramenta serve como um norte para o alunos que a usarem.
